# Project - AI Assistant for a Car Rental Company

A simple chatbot that helps a customer book a vehicle

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

/Users/huzaifa/projects/car-rental-ai-chatbot-v1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [4]:
vehicle_id_to_name = {
    "Toyota Coaster": "1b764193-d727-4ad5-a5bf-9c6ffc6bd854",
    "Mercedes G-Wagon": "23bd3571-a8d1-4bda-92cc-e769a00fa9cc",
    "Toyota Hiace 2018": "2e106f4e-1f32-4f33-886b-2be7fa62379b",
    "Toyota Land Cruiser 2020": "3a515596-abef-4484-9076-e49e71dce378",
    "Mercedes Sprinter": "7031459c-fde3-4625-bf04-d15734f8d2da",
    "Lexus GX 460": "71f073e1-4c04-4330-acd5-740735386405",
    "Toyota Hilux": "7b944cba-9110-44ba-ad47-2581990d8ee1",
    "Lexus LX 470": "a52c194a-411b-4d4b-8147-9ed45309f6eb",
    "Toyota Camry": "acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5",
    "Toyota Prado": "b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130",
    "Hyundai H1": "b9e1cc17-9620-4b04-8fad-614a014138a2",
    "Toyota Land Cruiser": "f089f58c-aed3-44e1-b7fb-2ed7b4b41281",
    "Toyota Hiace": "fb1f6d9d-2121-4a4b-a0c6-c9919414103e"
}

vehicle_names = ["Toyota Coaster", "Mercedes G-Wagon", "Toyota Hiace 2018", "Toyota Land Cruiser 2020", "Mercedes Sprinter", "Lexus GX 460", "Toyota Hilux", "Lexus LX 470", "Toyota Camry", "Toyota Prado", "Hyundai H1", "Toyota Land Cruiser", "Toyota Hiace"]
service_types = ["airport_pickup", "hourly_rate", "full_day_8hr", "full_24hr", "border_states_roundtrip"]

In [5]:
# New system message

system_message = f"""You are a helpful assistant for Exclusive Car Rentals. Their website is \
https://www.ecr.ng. Exclusive Car Rentals is a vehicle rental company. You are to \
help the user find the best vehicle for their needs and guide them through the booking process. The \
booking process is as follows: The user select the desired service, then select the vehicle, then \
enter their details. They will then get contacted by a customer service representative.
Always make sure the conversation is polite and friendly \
and stays within the scope of the website. If you do not know the answer, just say so.
Here is a list of available vehicles as-is in the DB:
{vehicle_names}
Use the vehicle name as-is when searching for a vehicle price, and 
The cost of renting a vehicle depends on the requested service type. Each vehicle has these 4 \
service types: {service_types}
Use a service type name as-is when searching using a service type. If a user doesn't specify a \
service type, search by airport_pickup.
"""

In [8]:
VEHICLE_DB = "vehicle.db"
VEHICLE_PRICES_DB = "vehicle_prices.db"

In [ ]:
with sqlite3.connect(VEHICLE_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS vehicles (id UUID PRIMARY KEY, name TEXT, type TEXT, image TEXT, seats INT4, transmission TEXT, fuel TEXT, features TEXT, available BOOL, created_at TIMESTAMPTZ, updated_at TIMESTAMPTZ, year INT4, popularity_status TEXT)")
    conn.commit()

with sqlite3.connect(VEHICLE_PRICES_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS vehicle_prices (id UUID PRIMARY KEY, vehicle_id UUID, service_type TEXT, price NUMERIC, is_available BOOL, created_at TIMESTAMPTZ, updated_at TIMESTAMPTZ)")
    conn.commit()

In [ ]:
# Populating the databases

with sqlite3.connect(VEHICLE_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("INSERT INTO vehicles (id, name, type, image, seats, transmission, fuel, features, available, created_at, updated_at, year, popularity_status) VALUES ('1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'Toyota Coaster', 'Coaster', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/67c595aa-1b32-465e-8493-58f69f8ce0dd.png', '30', 'Automatic', 'Petrol', null, 'true', '2025-10-21 11:49:43.594191+00', '2025-10-21 11:49:43.92327+00', '2023', 'normal'), ('23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'Mercedes G-Wagon', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/b2c28cdc-d6a1-42e1-8a01-4d4bf9203718.png', '5', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | luxurious leather interior | advanced infotainment system | adaptive suspension | premium sound system', 'false', '2025-10-04 08:34:00.110835+00', '2025-10-21 13:18:20.633985+00', '2020', 'unpopular'), ('2e106f4e-1f32-4f33-886b-2be7fa62379b', 'Toyota Hiace 2018', 'Bus', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/35de8f12-55c4-4329-be5e-6b8848fcc2d4.png', '18', 'Automatic', 'Petrol', '18-passenger seating | durable interior design | powerful diesel engine | rear air conditioning | spacious luggage area', 'true', '2025-10-21 11:40:46.761689+00', '2025-10-21 11:41:15.234146+00', '2018', 'normal'), ('3a515596-abef-4484-9076-e49e71dce378', 'Toyota Land Cruiser 2020', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/f9d711fa-8d95-4b45-82c6-cef0a9a7e695.png', '7', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | leather interior | touchscreen infotainment system | adaptive cruise control | multi-zone climate control', 'true', '2025-10-21 12:43:41.640437+00', '2025-10-21 12:45:54.198264+00', '2020', 'normal'), ('7031459c-fde3-4625-bf04-d15734f8d2da', 'Mercedes Sprinter', 'Bus', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/440e17af-ca36-44ab-aa4d-28eeba963cca.png', '15', 'Automatic', 'Petrol', '15-passenger capacity | advanced safety assist systems | touchscreen infotainment | rearview camera | climate control system', 'true', '2025-10-21 13:09:54.299733+00', '2025-10-21 13:10:54.837871+00', '2020', 'unpopular'), ('71f073e1-4c04-4330-acd5-740735386405', 'Lexus GX 460', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/4c699a67-aaf9-430b-b0ba-b8c99d7cd509.png', '7', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | premium leather interior | touchscreen infotainment system | adaptive cruise control | multi-zone climate control', 'true', '2025-10-04 08:34:00.110835+00', '2025-10-21 13:27:57.957887+00', '2020', 'popular'), ('7b944cba-9110-44ba-ad47-2581990d8ee1', 'Toyota Hilux', 'Pickup', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/75da0e88-ef8b-495c-bb8e-c29558663579.png', '5', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | touchscreen infotainment system | rearview camera | cruise control | advanced safety airbags', 'true', '2025-10-21 09:42:22.639841+00', '2025-10-21 09:43:00.683463+00', '2020', 'normal'), ('a52c194a-411b-4d4b-8147-9ed45309f6eb', 'Lexus LX 470', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/b8269d48-745c-4e7f-86ad-4e0e09cf5757.png', '5', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | premium leather seats | adaptive suspension system | touchscreen infotainment display | multi-zone climate control', 'true', '2025-10-21 13:22:24.496169+00', '2025-10-21 13:22:56.527784+00', '2020', 'normal'), ('acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'Toyota Camry', 'Saloon', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/c3829cdc-40f9-435d-95f7-8e99d99da59c.png', '5', 'Automatic', 'Petrol', 'Leather seats | touchscreen display | rearview camera | keyless entry | dual-zone climate control', 'true', '2025-10-21 08:50:47.613058+00', '2025-10-21 08:50:48.24966+00', '2017', 'normal'), ('b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'Toyota Prado', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/2oryjklhpj9-1759598265313.jpg', '7', 'Automatic', 'Petrol', '4WD | Climate Control | Reverse Camera | Bluetooth | Cruise Control', 'true', '2025-10-04 08:34:00.110835+00', '2025-10-23 12:50:25.004897+00', '2020', 'popular'), ('b9e1cc17-9620-4b04-8fad-614a014138a2', 'Hyundai H1', 'Bus', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/9cd4f174-459d-45a8-bf43-fbbaf0964b1e.png', '11', 'Automatic', 'Petrol', '11-passenger capacity | dual air conditioning | sliding rear doors | touchscreen infotainment system | rear parking sensors', 'true', '2025-10-21 12:58:03.683027+00', '2025-10-21 12:59:00.223559+00', '2017', 'normal'), ('f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'Toyota Land Cruiser', 'SUV', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/eb788746-7a0f-4fbc-91ef-94edeb294fa2.png', '7', 'Automatic', 'Petrol', 'Four-wheel drive (4WD) | leather seats | touchscreen infotainment system | adaptive cruise control | multi-terrain select system', 'true', '2025-10-04 08:34:00.110835+00', '2025-10-21 12:50:05.54979+00', '2023', 'popular'), ('fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'Toyota Hiace', 'Bus', 'https://znxuojydmexxlfapmxdl.supabase.co/storage/v1/object/public/suv-images/64d9fe73-fa14-4df3-bb01-eafb3a15cacc.png', '18', 'Automatic', 'Petrol', '18-passenger capacity | rear air conditioning vents | touchscreen infotainment system | advanced safety airbags | rear parking sensors', 'true', '2025-10-21 11:31:16.715027+00', '2025-10-21 11:32:20.735407+00', '2023', 'normal')")
    conn.commit()

with sqlite3.connect(VEHICLE_PRICES_DB) as conn:
    cursor = conn.cursor()
    cursor.execute("INSERT INTO vehicle_prices (id, vehicle_id, service_type, price, is_available, created_at, updated_at) VALUES ('00c8b4ed-a262-49f1-b2d6-88eddc8d1ab9', 'fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'border_states_roundtrip', '350000', 'true', '2025-10-21 11:31:17.32466+00', '2025-10-21 11:32:20.735407+00'), ('023f9c3f-00c1-4b0c-b2a7-b5c3d6e9a987', 'acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'hourly_rate', '25000', 'true', '2025-10-21 08:50:48.24966+00', '2025-10-21 08:50:48.24966+00'), ('02866912-a044-4a11-a6a5-a630d6ec7182', '7b944cba-9110-44ba-ad47-2581990d8ee1', 'border_states_roundtrip', '350000', 'true', '2025-10-21 09:42:23.12811+00', '2025-10-21 09:43:00.683463+00'), ('036a9e85-b110-4264-b683-25aa618e4ff1', 'fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'full_day_8hr', '200000', 'true', '2025-10-21 11:31:17.32466+00', '2025-10-21 11:32:20.735407+00'), ('05f9c23b-78ce-4e28-b172-0661329486ef', 'b9e1cc17-9620-4b04-8fad-614a014138a2', 'full_day_8hr', '150000', 'true', '2025-10-21 12:58:04.069918+00', '2025-10-21 12:59:00.223559+00'), ('10fe81e0-07b4-43dd-9db9-f295a9add3b4', 'f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'airport_pickup', '100000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 12:50:05.54979+00'), ('12eeaed3-667c-48d1-9efc-d6c34a581c31', 'b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'border_states_roundtrip', '450000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-23 12:50:25.004897+00'), ('1673c244-5d04-4965-9199-97a27e96096f', '23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'hourly_rate', '200000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:18:20.633985+00'), ('1cdc618d-48f5-45d8-8f08-2974005ebd67', '3a515596-abef-4484-9076-e49e71dce378', 'full_24hr', '700000', 'true', '2025-10-21 12:43:41.981382+00', '2025-10-21 12:45:54.198264+00'), ('23068dfb-63d5-44d3-9bec-f732a9da9eda', '23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'airport_pickup', '200000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:18:20.633985+00'), ('2bdab830-d354-4e7c-bb76-52777449d2a0', 'a52c194a-411b-4d4b-8147-9ed45309f6eb', 'full_24hr', '700000', 'true', '2025-10-21 13:22:24.809532+00', '2025-10-21 13:22:56.527784+00'), ('2d0d8f62-8e44-4710-9a91-0da01861c7c4', 'f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'hourly_rate', '100000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 12:50:05.54979+00'), ('3371a10e-386f-42ef-b388-f125dee8dd73', 'acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'airport_pickup', '30000', 'true', '2025-10-21 08:50:48.24966+00', '2025-10-21 08:50:48.24966+00'), ('341d1634-c7b1-43b3-8739-c8ba0533cd09', 'acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'border_states_roundtrip', '250000', 'true', '2025-10-21 08:50:48.24966+00', '2025-10-21 08:50:48.24966+00'), ('3692cb23-7e11-40ef-b53b-ffe1bdb97c16', 'a52c194a-411b-4d4b-8147-9ed45309f6eb', 'airport_pickup', '100000', 'true', '2025-10-21 13:22:24.809532+00', '2025-10-21 13:22:56.527784+00'), ('3cfdf5b6-9f68-4d0e-bf22-8d37a56b9c1e', 'f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'full_day_8hr', '400000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 12:50:05.54979+00'), ('4122b22b-61f0-47e3-873c-1f9283bf0b81', '1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'full_day_8hr', '250000', 'true', '2025-10-21 11:49:43.92327+00', '2025-10-21 11:49:43.92327+00'), ('4482e462-885c-495c-8ca8-f196bee5bb8a', '1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'hourly_rate', '100000', 'true', '2025-10-21 11:49:43.92327+00', '2025-10-21 11:49:43.92327+00'), ('4a76ba1b-4b20-4dae-932e-c62fba2bc7c5', '7031459c-fde3-4625-bf04-d15734f8d2da', 'full_day_8hr', '1000000', 'true', '2025-10-21 13:09:54.539419+00', '2025-10-21 13:10:54.837871+00'), ('535b6e0f-86c7-4d4c-88fd-2a81abd068b4', '2e106f4e-1f32-4f33-886b-2be7fa62379b', 'hourly_rate', '50000', 'true', '2025-10-21 11:40:47.016472+00', '2025-10-21 11:41:15.234146+00'), ('55d81dff-91ae-4dc0-9597-605fa4c4dfcf', '71f073e1-4c04-4330-acd5-740735386405', 'hourly_rate', '80000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:27:57.957887+00'), ('59ce2855-08bf-4c8a-ae2b-9c6eada3f9a6', '1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'border_states_roundtrip', '500000', 'true', '2025-10-21 11:49:43.92327+00', '2025-10-21 11:49:43.92327+00'), ('5a00488a-348d-4eb4-aa84-ca18790b7c9c', '2e106f4e-1f32-4f33-886b-2be7fa62379b', 'airport_pickup', '50000', 'true', '2025-10-21 11:40:47.016472+00', '2025-10-21 11:41:15.234146+00'), ('5c9e9d0e-c23d-427b-a9a8-12852eba9b3e', '2e106f4e-1f32-4f33-886b-2be7fa62379b', 'full_day_8hr', '200000', 'true', '2025-10-21 11:40:47.016472+00', '2025-10-21 11:41:15.234146+00'), ('687d8b8c-2cf0-4119-aad7-0b0a7fb6e507', 'b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'airport_pickup', '80000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-23 12:50:25.004897+00'), ('6918761d-bc36-47db-b994-2c6073190a20', '7b944cba-9110-44ba-ad47-2581990d8ee1', 'full_24hr', '400000', 'true', '2025-10-21 09:42:23.12811+00', '2025-10-21 09:43:00.683463+00'), ('6b52906f-b8b9-442a-b2f2-4c8bda84e066', 'b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'full_day_8hr', '230000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-23 12:50:25.004897+00'), ('8617cf67-beca-44d0-a837-02093f087980', 'a52c194a-411b-4d4b-8147-9ed45309f6eb', 'hourly_rate', '100000', 'true', '2025-10-21 13:22:24.809532+00', '2025-10-21 13:22:56.527784+00'), ('8b4ccea1-4a78-41ba-9e82-58c853df7daf', '7031459c-fde3-4625-bf04-d15734f8d2da', 'border_states_roundtrip', '1800000', 'true', '2025-10-21 13:09:54.539419+00', '2025-10-21 13:10:54.837871+00'), ('8fa04c74-966d-473b-8e3e-3e2456145720', 'f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'border_states_roundtrip', '600000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 12:50:05.54979+00'), ('97bce532-449f-40d6-839c-b978f3d7162a', '7b944cba-9110-44ba-ad47-2581990d8ee1', 'full_day_8hr', '200000', 'true', '2025-10-21 09:42:23.12811+00', '2025-10-21 09:43:00.683463+00'), ('97cfdfeb-98f8-4103-8d42-a7ed4df212e2', 'b9e1cc17-9620-4b04-8fad-614a014138a2', 'hourly_rate', '30000', 'true', '2025-10-21 12:58:04.069918+00', '2025-10-21 12:59:00.223559+00'), ('9844dabb-6803-489c-b929-84c8787fa747', '71f073e1-4c04-4330-acd5-740735386405', 'border_states_roundtrip', '500000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:27:57.957887+00'), ('a8564a7f-49eb-4f50-a1b7-e5adff6bc2b6', '7031459c-fde3-4625-bf04-d15734f8d2da', 'airport_pickup', '200000', 'true', '2025-10-21 13:09:54.539419+00', '2025-10-21 13:10:54.837871+00'), ('a9119bb3-bf17-4d6e-818a-323f5804bc46', 'acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'full_24hr', '240000', 'true', '2025-10-21 08:50:48.24966+00', '2025-10-21 08:50:48.24966+00'), ('ad12b531-d494-4e3e-8c7a-d316dbb0eee5', 'b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'hourly_rate', '80000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-23 12:50:25.004897+00'), ('ad81c56a-ee37-4de5-8780-05319a268382', '23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'border_states_roundtrip', '1800000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:18:20.633985+00'), ('b0d720a6-1da6-4195-b22c-cc3f2e3e34ed', '1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'airport_pickup', '100000', 'true', '2025-10-21 11:49:43.92327+00', '2025-10-21 11:49:43.92327+00'), ('b4c744f5-32bd-4d24-802e-ad0d55fe7db3', '71f073e1-4c04-4330-acd5-740735386405', 'airport_pickup', '80000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:27:57.957887+00'), ('b5b9222c-96c5-45a9-97db-57de1870e252', 'b9e1cc17-9620-4b04-8fad-614a014138a2', 'border_states_roundtrip', '250000', 'true', '2025-10-21 12:58:04.069918+00', '2025-10-21 12:59:00.223559+00'), ('b6c95093-3331-47e3-a664-c7e514f1514d', 'b9e1cc17-9620-4b04-8fad-614a014138a2', 'full_24hr', '300000', 'true', '2025-10-21 12:58:04.069918+00', '2025-10-21 12:59:00.223559+00'), ('b72e2d26-2d24-45b1-a986-1f38e7030779', '71f073e1-4c04-4330-acd5-740735386405', 'full_24hr', '500000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:27:57.957887+00'), ('bedd7423-2c65-4337-9208-3569d4f0dc12', 'fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'airport_pickup', '50000', 'true', '2025-10-21 11:31:17.32466+00', '2025-10-21 11:32:20.735407+00'), ('bfef6ea7-5eac-4cb2-a9ac-9399f9c283d7', '7b944cba-9110-44ba-ad47-2581990d8ee1', 'airport_pickup', '70000', 'true', '2025-10-21 09:42:23.12811+00', '2025-10-21 09:43:00.683463+00'), ('c08d0c70-41e4-4eb7-858e-8b236ca4c3b0', '3a515596-abef-4484-9076-e49e71dce378', 'hourly_rate', '100000', 'true', '2025-10-21 12:43:41.981382+00', '2025-10-21 12:45:54.198264+00'), ('c37fc6b5-e0e4-460f-b9c9-7ab1a8f73311', '1b764193-d727-4ad5-a5bf-9c6ffc6bd854', 'full_24hr', '500000', 'true', '2025-10-21 11:49:43.92327+00', '2025-10-21 11:49:43.92327+00'), ('c405f1af-3a4a-48c4-861a-0831a9f45417', '3a515596-abef-4484-9076-e49e71dce378', 'border_states_roundtrip', '600000', 'true', '2025-10-21 12:43:41.981382+00', '2025-10-21 12:45:54.198264+00'), ('c779b358-01b8-43c4-af02-137e273fd60d', 'b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130', 'full_24hr', '450000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-23 12:50:25.004897+00'), ('c9262152-440e-4cc1-8454-33f5ab291aba', 'fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'full_24hr', '400000', 'true', '2025-10-21 11:31:17.32466+00', '2025-10-21 11:32:20.735407+00'), ('d08b35ec-1d0e-46e9-97cb-7b7f26deee4c', 'b9e1cc17-9620-4b04-8fad-614a014138a2', 'airport_pickup', '40000', 'true', '2025-10-21 12:58:04.069918+00', '2025-10-21 12:59:00.223559+00'), ('d83b6a50-6e18-4200-b470-5533e739509f', 'a52c194a-411b-4d4b-8147-9ed45309f6eb', 'border_states_roundtrip', '700000', 'true', '2025-10-21 13:22:24.809532+00', '2025-10-21 13:22:56.527784+00'), ('df0f24e5-9e3d-4032-a7db-f18fced0abc4', 'f089f58c-aed3-44e1-b7fb-2ed7b4b41281', 'full_24hr', '700000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 12:50:05.54979+00'), ('e0c18c78-3212-4e57-a7d4-0412fd408cf3', '23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'full_24hr', '1500000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:18:20.633985+00'), ('e388474b-39e7-418e-a207-a1bbe7409138', '2e106f4e-1f32-4f33-886b-2be7fa62379b', 'full_24hr', '400000', 'true', '2025-10-21 11:40:47.016472+00', '2025-10-21 11:41:15.234146+00'), ('e59a567a-71ca-4155-878f-209b8dec14c1', '71f073e1-4c04-4330-acd5-740735386405', 'full_day_8hr', '250000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:27:57.957887+00'), ('eb331e8a-30e4-451e-82aa-5b44b5d17814', '7031459c-fde3-4625-bf04-d15734f8d2da', 'hourly_rate', '200000', 'true', '2025-10-21 13:09:54.539419+00', '2025-10-21 13:10:54.837871+00'), ('ebc605bd-694a-417d-b376-26306701435e', '3a515596-abef-4484-9076-e49e71dce378', 'full_day_8hr', '400000', 'true', '2025-10-21 12:43:41.981382+00', '2025-10-21 12:45:54.198264+00'), ('eee127c4-07aa-44d4-b296-890c48074402', '7b944cba-9110-44ba-ad47-2581990d8ee1', 'hourly_rate', '60000', 'true', '2025-10-21 09:42:23.12811+00', '2025-10-21 09:43:00.683463+00'), ('ef3ceebb-c2e7-412e-ad86-99892cded29d', '2e106f4e-1f32-4f33-886b-2be7fa62379b', 'border_states_roundtrip', '350000', 'true', '2025-10-21 11:40:47.016472+00', '2025-10-21 11:41:15.234146+00'), ('f208f8bb-3df1-45c2-a177-e7bc5f5ffb4d', '7031459c-fde3-4625-bf04-d15734f8d2da', 'full_24hr', '1500000', 'true', '2025-10-21 13:09:54.539419+00', '2025-10-21 13:10:54.837871+00'), ('f2fcf711-ee99-4f72-9d05-57f2de75e067', 'a52c194a-411b-4d4b-8147-9ed45309f6eb', 'full_day_8hr', '500000', 'true', '2025-10-21 13:22:24.809532+00', '2025-10-21 13:22:56.527784+00'), ('f4f4a8b1-f7df-4d1b-af99-df5dc8081dd8', 'fb1f6d9d-2121-4a4b-a0c6-c9919414103e', 'hourly_rate', '50000', 'true', '2025-10-21 11:31:17.32466+00', '2025-10-21 11:32:20.735407+00'), ('f761d0ad-c209-484c-9080-709ef73a80f2', '23bd3571-a8d1-4bda-92cc-e769a00fa9cc', 'full_day_8hr', '1000000', 'true', '2025-10-11 19:49:38.258255+00', '2025-10-21 13:18:20.633985+00'), ('f7cecf0d-a0e1-435e-b0db-c07e0c93ce69', '3a515596-abef-4484-9076-e49e71dce378', 'airport_pickup', '100000', 'true', '2025-10-21 12:43:41.981382+00', '2025-10-21 12:45:54.198264+00'), ('f82e07a1-a9b5-4e68-b24b-706b05e755a4', 'acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5', 'full_day_8hr', '120000', 'true', '2025-10-21 08:50:48.24966+00', '2025-10-21 08:50:48.24966+00');")
    conn.commit()

In [6]:
def get_vehicle_price(vehicle_name, service_type):
    vehicle_id = vehicle_id_to_name[vehicle_name]
    print(f"Checking the DB for vehicle ID {vehicle_id} and service type {service_type}", flush=True)
    with sqlite3.connect(VEHICLE_PRICES_DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM vehicle_prices WHERE vehicle_id = ? AND service_type = ?", (vehicle_id, service_type))
        result = cursor.fetchone()
        return f"The price for its {service_type} is {result[0]}" if result else "No price found"

In [9]:
get_vehicle_price("Toyota Coaster", "border_states_roundtrip")

Checking the DB for vehicle ID 1b764193-d727-4ad5-a5bf-9c6ffc6bd854 and service type border_states_roundtrip


'The price for its border_states_roundtrip is 500000'

In [10]:
vehicle_price_function = {
    "name": "get_vehicle_price",
    "description": "Get the rental price for a specific vehicle and service type. Use this when the customer asks about pricing for a vehicle rental.",
    "parameters": {
        "type": "object",
        "properties": {
            "vehicle_name": {
                "type": "string",
                "description": "The name of the vehicle the customer wants to rent (e.g., 'Toyota Coaster', 'Mercedes G-Wagon', 'Toyota Hiace'). Use the exact name from the available vehicles."
            },
            "service_type": {
                "type": "string",
                "description": "The type of rental service requested. Must be one of: 'hourly_rate' (hourly rental), 'full_day_8hr' (8-hour full day rental), 'full_24hr' (24-hour rental), 'airport_pickup' (airport pickup service), or 'border_states_roundtrip' (round trip to border states)."
            }
        },
        "required": ["vehicle_name", "service_type"],
        "additionalProperties": False
    }
}

In [11]:
tools = [{"type": "function", "function": vehicle_price_function}]

In [12]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [13]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_vehicle_price":
            arguments = json.loads(tool_call.function.arguments)
            vehicle_name = arguments.get("vehicle_name")
            service_type = arguments.get("service_type")
            price_info = get_vehicle_price(vehicle_name, service_type)
            responses.append({
                "role": "tool",
                "content": price_info,
                "tool_call_id": tool_call.id
            })
    return responses

In [24]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
